# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
import plotly
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Load data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
df_imm = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
feats_imm = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
df_ld_imm = df_imm['Subject ID'].value_counts().to_frame()
df_imm['Is longitudinal?'] = False
df_imm.loc[df_imm['Subject ID'].isin(df_ld_imm.index[df_ld_imm['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_imm = df_imm.loc[(df_imm['Status'] == 'Control') & (df_imm['Is longitudinal?'] == True), :]
df_imm.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_imm['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)

dnam_suffix = "_harm"
pheno = pd.read_excel(f"{path}/pheno.xlsx", index_col="index")
pheno.index.name = "index"
pheno.drop(["I64_old", "I1_duplicate"], inplace=True)

betas = pd.read_pickle(f"{path}/betas{dnam_suffix}.pkl")
feats_dnam = betas.columns.values
df_dnam = pd.merge(pheno, betas, left_index=True, right_index=True)

df_ld_dnam = df_dnam['Subject_ID'].value_counts().to_frame()
df_dnam['Is longitudinal?'] = False
df_dnam.loc[df_dnam['Subject_ID'].isin(df_ld_dnam.index[df_ld_dnam['Subject_ID'] > 1].values), 'Is longitudinal?'] = True
df_dnam = df_dnam.loc[(df_dnam['Status'] == 'Control') & (df_dnam['Is longitudinal?'] == True), :]
df_dnam.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_dnam['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)

pace = "DunedinPACE"
epi_age_types = {
    f"DNAmAgeHannum{dnam_suffix}": "Hannum",
    f"DNAmAge{dnam_suffix}": "Horvath",
    f"DNAmAgeSkinBloodClock{dnam_suffix}": "SkinBloodAge",
    f"DNAmPhenoAge{dnam_suffix}": "PhenoAge",
    f"DNAmGrimAge{dnam_suffix}": "GrimAge",
    "PCHannum": "PC-Hannum",
    "PCHorvath1": "PC-Horvath",
    "PCHorvath2": "PC-SkinBloodAge",
    "PCPhenoAge": "PC-PhenoAge",
    "PCGrimAge": "PC-GrimAge",
    "mPACE": pace
}
df_dnam.rename(columns=epi_age_types, inplace=True)
epi_ages = list(epi_age_types.values())
for x in epi_ages:
    if x != pace:
        df_dnam[f"{x}Acc"] = df_dnam[x] - df_dnam['Age']

expts = {
    '2_points': ['T0', 'T1'],
    '3_points': ['T0', 'T1', 'T2'],
    '4_points': ['T0', 'T1', 'T2', 'T3'],
}

path_save = f"{path}/special/058_unn_longitudinal_controls"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

# DNAm processing

In [ ]:
epi_est_plots = {
    'Hannum': {
        'col_id': 0,
        'row_id': 0,
        'title': 'Hannum',
        'y_label': 'Age Acceleration',
        'x_label': ''
    },
    'Horvath': {
        'col_id': 1,
        'row_id': 0,
        'title': 'Horvath',
        'y_label': '',
        'x_label': ''
    },
    'SkinBloodAge': {
        'col_id': 2,
        'row_id': 0,
        'title': 'SkinBloodAge',
        'y_label': '',
        'x_label': ''
    },
    'PhenoAge': {
        'col_id': 3,
        'row_id': 0,
        'title': 'PhenoAge',
        'y_label': '',
        'x_label': ''
    },
    'GrimAge': {
        'col_id': 4,
        'row_id': 0,
        'title': 'GrimAge',
        'y_label': '',
        'x_label': ''
    },
    'DunedinPACE': {
        'col_id': 5,
        'row_id': 0,
        'title': 'DunedinPACE',
        'y_label': '',
        'x_label': 'Time'
    },
    'PC-Hannum': {
        'col_id': 0,
        'row_id': 1,
        'title': 'PC-Hannum',
        'y_label': 'Age Acceleration',
        'x_label': 'Time'
    },
    'PC-Horvath': {
        'col_id': 1,
        'row_id': 1,
        'title': 'PC-Horvath',
        'y_label': '',
        'x_label': 'Time'
    },
    'PC-SkinBloodAge': {
        'col_id': 2,
        'row_id': 1,
        'title': 'PC-SkinBloodAge',
        'y_label': '',
        'x_label': 'Time'
    },
    'PC-PhenoAge': {
        'col_id': 3,
        'row_id': 1,
        'title': 'PC-PhenoAge',
        'y_label': '',
        'x_label': 'Time'
    },
    'PC-GrimAge': {
        'col_id': 4,
        'row_id': 1,
        'title': 'PC-GrimAge',
        'y_label': '',
        'x_label': 'Time'
    },
}

for expt_name, time_points in expts.items():
    path_curr = f"{path_save}/epi/{expt_name}"
    pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)
    
    samples_with_times = set.intersection(*[set(df_dnam.loc[df_dnam['Time'] == x, 'Subject_ID'].unique()) for x in time_points])
    
    colors_xkcd = list(matplotlib.colors.XKCD_COLORS.values())
    colors_samples = {x: px.colors.qualitative.Light24[x_id] for x_id, x in enumerate(samples_with_times)}
    
    df_expt = df_dnam[(df_dnam['Time'].isin(time_points)) & (df_dnam['Subject_ID'].isin(samples_with_times))]
    df_expt.sort_values(["Time"], ascending=[True], inplace=True)
    
    
    n_rows = 3
    n_cols = len(epi_est_plots)
    fig_width = 24
    fig_height = 12
    ptp_shift = 0.05
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={})
    sns.set_theme(style='whitegrid')
    
    legend_handles = [
        mlines.Line2D(
            [],
            [],
            marker='o',
            linestyle='None',
            markeredgecolor='k',
            markerfacecolor=colors_samples[sample],
            markersize=10,
            label=sample)
        for sample in samples_with_times
    ]
    for epi_est_id, epi_est in enumerate(epi_est_plots):
        if epi_est == pace:
            col = epi_est
            is_legend = True
        else:
            col = f"{epi_est}Acc"
            is_legend = False
        
        df_pivot = df_expt.pivot(index='Subject_ID', columns='Time', values=col).dropna()
        df_pivot['Sample Type'] = 'None'
        df_pivot_t = df_pivot.transpose()
        
        for sample in df_pivot_t:
            if df_pivot_t.loc[time_points, sample].is_monotonic_increasing:
                df_pivot.at[sample, 'Sample Type'] = 'Inc'
            elif df_pivot_t.loc[time_points, sample].is_monotonic_decreasing:
                df_pivot.at[sample, 'Sample Type'] = 'Dec'
        
        for row_id, sample_type in enumerate(['Inc', 'Dec', 'None']):
            samples_selected = df_pivot.index[df_pivot['Sample Type'] == sample_type].values
            sns.scatterplot(
                data=df_expt.loc[df_expt['Subject_ID'].isin(samples_selected), :],
                x='Time',
                y=col,
                hue='Subject_ID',
                style='Subject_ID',
                edgecolor="k",
                linewidth=0.001,
                palette=colors_samples,
                hue_order=list(colors_samples.keys()),
                alpha=0.75,
                s=100,
                legend=False,
                ax=axs[row_id, epi_est_id]
            )
            sns.lineplot(
                data=df_expt.loc[df_expt['Subject_ID'].isin(samples_selected), :],
                x='Time',
                y=col,
                hue='Subject_ID',
                palette=colors_samples,
                hue_order=list(colors_samples.keys()),
                legend=False,
                ax=axs[row_id, epi_est_id]
            )
            if epi_est_id == 0:
                axs[row_id, epi_est_id].set_ylabel('Age acceleration')
            else:
                axs[row_id, epi_est_id].set_ylabel('')
            if row_id == 2:
                axs[row_id, epi_est_id].set_xlabel('Time')
            else:
                axs[row_id, epi_est_id].set_xlabel('')
                axs[row_id, epi_est_id].set_xticklabels([])
            if row_id == 0:
                axs[row_id, epi_est_id].set_title(f"{epi_est}\n\nIncreased", fontsize = 12)
            elif row_id == 1:
                axs[row_id, epi_est_id].set_title(f"Decreased", fontsize = 12)
            elif row_id == 2:
                axs[row_id, epi_est_id].set_title(f"Non-monotonic", fontsize = 12)
    
    fig.tight_layout()
    fig.legend(handles=legend_handles, loc="lower center", bbox_to_anchor=(.5, 1), ncol=len(samples_with_times) // 2 + 1, frameon=False)
    plt.savefig(f"{path_curr}/groups.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_curr}/groups.pdf", bbox_inches='tight')
    plt.close(fig)
    
    n_rows = 2
    n_cols = 6
    fig_width = 20
    fig_height = 8
    ptp_shift = 0.05
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={})
    sns.set_theme(style='whitegrid')
    
    df_stat = pd.DataFrame(index=list(epi_est_plots.keys()), columns=['pval', 'pval_fdr_bh'])
    for epi_est, params in epi_est_plots.items():
        if epi_est == pace:
            col = epi_est
            is_legend = True
        else:
            col = f"{epi_est}Acc"
            is_legend = False
        
        df_pivot = df_expt.pivot(index='Subject_ID', columns='Time', values=col).dropna()
        # df_melt = df_pivot.melt(value_vars=time_points, var_name='Time', value_name=col, ignore_index=False)

        sns.scatterplot(
            data=df_expt,
            x='Time',
            y=col,
            hue='Subject_ID',
            style='Subject_ID',
            edgecolor="k",
            linewidth=0.001,
            palette=colors_samples,
            hue_order=list(colors_samples.keys()),
            alpha=0.75,
            s=100,
            legend=is_legend,
            ax=axs[params['row_id'], params['col_id']]
        )
        if is_legend:
            axs[params['row_id'], params['col_id']].get_legend().remove()
            handles, labels = axs[params['row_id'], params['col_id']].get_legend_handles_labels()
            for ha in handles:
                ha.set_edgecolor("black")
                ha.set_linewidth(0.01)
                ha.set_sizes([100])
            if len(df_expt['Subject_ID'].unique()) > 15:
                legend_ncol = len(df_expt['Subject_ID'].unique()) // 2
            else:
                legend_ncol = len(df_expt['Subject_ID'].unique())
            fig.legend(handles, labels, loc='lower center', ncol=legend_ncol, bbox_to_anchor=(0.5, 1.0), frameon=False, fontsize='large')
        sns.lineplot(
            data=df_expt,
            x='Time',
            y=col,
            hue='Subject_ID',
            palette=colors_samples,
            hue_order=list(colors_samples.keys()),
            legend=False,
            ax=axs[params['row_id'], params['col_id']]
        )
        axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'])
        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
        if params['x_label'] == '':
            axs[params['row_id'], params['col_id']].set_xticklabels([])
        
        if len(time_points) == 2:
            res = wilcoxon(
                x=df_pivot.loc[:, 'T0'].values,
                y=df_pivot.loc[:, 'T1'].values,
                alternative='two-sided'
            )
            pval = res.pvalue
        else:
            res = friedmanchisquare(
                *[df_pivot.loc[:, x].values for x in time_points]
            )
            pval = res.pvalue
        df_stat.at[epi_est, "pval"] = pval
        
    _, df_stat.loc[:, "pval_fdr_bh"], _, _ = multipletests(df_stat.loc[:, "pval"], 0.05, method='fdr_bh')
    df_stat.sort_values([f"pval_fdr_bh"], ascending=[True], inplace=True)
    df_stat.to_excel(f"{path_curr}/stat.xlsx", index_label='Features')
    
    with pd.ExcelWriter(f'{path_curr}/table.xlsx', engine='xlsxwriter') as writer:
        df_pivot = df_expt.pivot(index='Subject_ID', columns='Time', values='Age').dropna()
        df_pivot.to_excel(writer, sheet_name='Age')
        for epi_est, params in epi_est_plots.items():
            if epi_est == pace:
                col = epi_est
            else:
                col = f"{epi_est}Acc"
            df_pivot = df_expt.pivot(index='Subject_ID', columns='Time', values=col).dropna()
            df_pivot.to_excel(writer, sheet_name=col)
            pval = df_stat.at[epi_est, "pval_fdr_bh"]
            label = r'p-value: ' + f"{pval:0.1e}"
            axs[params['row_id'], params['col_id']].set_title(f"{params['title']}\n{label}", fontsize = 16)
        axs[n_rows - 1, n_cols - 1].set_xlabel('')
        axs[n_rows - 1, n_cols - 1].axis('off')
        
    fig.tight_layout()    
    plt.savefig(f"{path_curr}/pointplot.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_curr}/pointplot.pdf", bbox_inches='tight')
    plt.close(fig)
                

# Immunology processing

In [ ]:
for expt_name, time_points in expts.items():
    samples_with_times = set.intersection(*[set(df_imm.loc[df_imm['Time'] == x, 'Subject ID'].unique()) for x in time_points])
    print(f"{expt_name}: {len(samples_with_times)}")
    
    if len(samples_with_times) > 0:
        
        col = 'SImAge acceleration'
    
        path_curr = f"{path_save}/imm/{expt_name}"
        pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)
        
        colors_xkcd = list(matplotlib.colors.XKCD_COLORS.values())
        colors_samples = {x: px.colors.qualitative.Light24[x_id] for x_id, x in enumerate(samples_with_times)}
        
        df_expt = df_imm[(df_imm['Time'].isin(time_points)) & (df_imm['Subject ID'].isin(samples_with_times))]
        df_expt.sort_values(["Time"], ascending=[True], inplace=True)
        
        n_rows = 3
        n_cols = 1
        fig_width = 6
        fig_height = 12
        
        fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={})
        sns.set_theme(style='whitegrid')
        
        legend_handles = [
            mlines.Line2D(
                [],
                [],
                marker='o',
                linestyle='None',
                markeredgecolor='k',
                markerfacecolor=colors_samples[sample],
                markersize=10,
                label=sample)
            for sample in samples_with_times
        ]
        
        df_pivot = df_expt.pivot(index='Subject ID', columns='Time', values=col).dropna()
        df_pivot['Sample Type'] = 'None'
        df_pivot_t = df_pivot.transpose()
        
        for sample in df_pivot_t:
            if df_pivot_t.loc[time_points, sample].is_monotonic_increasing:
                df_pivot.at[sample, 'Sample Type'] = 'Inc'
            elif df_pivot_t.loc[time_points, sample].is_monotonic_decreasing:
                df_pivot.at[sample, 'Sample Type'] = 'Dec'
        
        for row_id, sample_type in enumerate(['Inc', 'Dec', 'None']):
            samples_selected = df_pivot.index[df_pivot['Sample Type'] == sample_type].values
            sns.scatterplot(
                data=df_expt.loc[df_expt['Subject ID'].isin(samples_selected), :],
                x='Time',
                y=col,
                hue='Subject ID',
                style='Subject ID',
                edgecolor="k",
                linewidth=0.001,
                palette=colors_samples,
                hue_order=list(colors_samples.keys()),
                alpha=0.75,
                s=100,
                legend=False,
                ax=axs[row_id]
            )
            sns.lineplot(
                data=df_expt.loc[df_expt['Subject ID'].isin(samples_selected), :],
                x='Time',
                y=col,
                hue='Subject ID',
                palette=colors_samples,
                hue_order=list(colors_samples.keys()),
                legend=False,
                ax=axs[row_id]
            )
            axs[row_id].set_ylabel('SImAge acceleration')
            if row_id == 2:
                axs[row_id].set_xlabel('Time')
            else:
                axs[row_id].set_xlabel('')
                axs[row_id].set_xticklabels([])
            if row_id == 0:
                axs[row_id].set_title(f"Increased", fontsize = 12)
            elif row_id == 1:
                axs[row_id].set_title(f"Decreased", fontsize = 12)
            elif row_id == 2:
                axs[row_id].set_title(f"Non-monotonic", fontsize = 12)
        
        fig.tight_layout()
        fig.legend(handles=legend_handles, loc="lower center", bbox_to_anchor=(.5, 1), ncol=5, frameon=False)
        plt.savefig(f"{path_curr}/groups.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_curr}/groups.pdf", bbox_inches='tight')
        plt.close(fig)
        
        fig, ax = plt.subplots(figsize=(6, 4))
        sns.set_theme(style='whitegrid')
        
        df_pivot = df_expt.pivot(index='Subject ID', columns='Time', values=col).dropna()
        # df_melt = df_pivot.melt(value_vars=time_points, var_name='Time', value_name=col, ignore_index=False)
    
        sns.scatterplot(
            data=df_expt,
            x='Time',
            y=col,
            hue='Subject ID',
            style='Subject ID',
            edgecolor="k",
            linewidth=0.001,
            palette=colors_samples,
            hue_order=list(colors_samples.keys()),
            alpha=0.75,
            s=100,
            legend=True,
            ax=ax
        )
        ax.get_legend().remove()
        handles, labels = ax.get_legend_handles_labels()
        for ha in handles:
            ha.set_edgecolor("black")
            ha.set_linewidth(0.01)
            ha.set_sizes([100])
        legend_ncol = 6
        fig.legend(handles, labels, loc='lower center', ncol=legend_ncol, bbox_to_anchor=(0.5, 1.0), frameon=False)
        sns.lineplot(
            data=df_expt,
            x='Time',
            y=col,
            hue='Subject ID',
            palette=colors_samples,
            hue_order=list(colors_samples.keys()),
            legend=False,
            ax=ax
        )
        
        if len(time_points) == 2:
            res = wilcoxon(
                x=df_pivot.loc[:, 'T0'].values,
                y=df_pivot.loc[:, 'T1'].values,
                alternative='two-sided'
            )
            pval = res.pvalue
        else:
            res = friedmanchisquare(
                *[df_pivot.loc[:, x].values for x in time_points]
            )
            pval = res.pvalue
        label = r'p-value: ' + f"{pval:0.1e}"
        ax.set_title(f"{label}", fontsize = 16)
        fig.tight_layout()    
        plt.savefig(f"{path_curr}/pointplot.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_curr}/pointplot.pdf", bbox_inches='tight')
        plt.close(fig)
        
        with pd.ExcelWriter(f'{path_curr}/table.xlsx', engine='xlsxwriter') as writer:
            df_pivot = df_expt.pivot(index='Subject ID', columns='Time', values='Age').dropna()
            df_pivot.to_excel(writer, sheet_name='Age')
            df_pivot = df_expt.pivot(index='Subject ID', columns='Time', values=col).dropna()
            df_pivot.to_excel(writer, sheet_name=col)